# PART A - SPOTIFY
### Group 1:
Zachary Wilkerson, Tara Bode, Hankun Li

### Some things I want to know:
- How many times did a track stay in the daily global top 200 list in 2021?
- How many times did an artist stay in the daily global top 200 list in 2021?

In [29]:
import time
from selenium import webdriver
import pandas as pd
import regex as re

options = webdriver.ChromeOptions();
options.add_argument("disable-geolocation");
options.add_argument("disable-notifications");

driver = webdriver.Chrome(r'/Users/zwilkerson/Desktop/UF/ACG7849 - Web Crawling and Textual Analysis/selenium_files/chromedriver',options=options)

In [30]:
# FUNCTION: Injects jQuery
def jq_injector(driver):
    with open(r'/Users/zwilkerson/Desktop/UF/ACG7849 - Web Crawling and Textual Analysis/selenium_files/jquery.js', 'r') as jquery_js:
        driver.execute_script( jquery_js.read() ) 

# FUNCTION: Loads a website using Selenium and injects jQuery
def website_loader(driver,site):
    driver.get(site)
    jq_injector(driver)

In [31]:
website_loader(driver,"https://spotifycharts.com/regional/")

In [32]:
# FUNCTION: Populates a dictionary
def dict_adder(key,dic):
    try:
        dic[key] += 1;
    except:
        dic[key] = 1;
    return dic

# FUNCTION: NAVIGATES TO PAGE OF NEW DATE
def date_navigation(driver,index):
    if index == -1: # STAY ON CURRENT PAGE IF BEGINNING OF LOOP
        get_date = '''
        date = $('.responsive-select')[2].innerText
        return date
        '''
    else: # MOVE TO PAGE OF NEW DATE
        get_date = '''
        button = $('.responsive-select')[2]
        button.click()
        next_date = $('div.responsive-select.active ul li[class="selected"]').index()
        new_button = $('div.responsive-select.active ul li')[next_date+1]
        date = new_button.innerText
        new_button.click()
        return date
        '''   
    date = driver.execute_script(get_date)
    return date
    
# FUNCTION: PULLS DATA FROM SITE
def data_pulling(driver):
    data_per_day = '''
    elements = $("td.chart-table-track")
    all_data = []
    elements.each( function(i,el) { all_data.push( el.innerText )  }   )
    return all_data;
    '''
    info = driver.execute_script(data_per_day)
    # COULD ADJUST TO OBTAIN POSITION IN TABLE AND NUMBER OF STREAMS IF DESIRED...
        # td.chart-table-position
        # td.chart-table-streams
    return info

In [33]:
song_dict = {};
artist_dict = {};

date_index = -1;
date = date_navigation(driver,date_index)

#while re.search(r'05/[0-9]{2}/2021',date): # WHILE DATE IS IN MAY 2021 (FOR TESTING)
while re.search(r'[0-9]{2}/[0-9]{2}/2021',date): # WHILE DATE IS IN 2021
    info = data_pulling(driver) # PULL DATA

    for item in range(len(info)): # SPLIT INFO TO GET SONG AND ARTIST
        song_artist_split = info[item].split(' by ')
        dict_adder(song_artist_split[0],song_dict)
        dict_adder(song_artist_split[1],artist_dict)
    
    date_index +=1;
    date = date_navigation(driver,date_index) # MOVE TO NEXT DATE

driver.close()
        

## Let's take a look at our data!
Uncomment and run the following lines if you'd like. 

If not, take a look at the printed top artists and songs!  

In [26]:
#artist_dict

In [27]:
#song_dict

In [34]:
# SEE: https://careerkarma.com/blog/python-sort-a-dictionary-by-value/#:~:text=To%20sort%20a%20dictionary%20by%20value%20in%20Python%20you%20can,Dictionaries%20are%20unordered%20data%20structures.
top_artists = sorted(artist_dict.items(), key=lambda x: x[1], reverse=True)[:5]
top_songs = sorted(song_dict.items(), key=lambda x: x[1], reverse=True)[:5]

print('[Top 5 Artists]\n')
for i in top_artists:
    print('{} \t - \t {} occurances'.format(i[0], i[1]))

print('__________________________________________________________________________')
    
print('\n[Top 5 Songs]\n')
for i in top_songs:
    print('{} \t - \t {} occurances'.format(i[0], i[1]))

[Top 5 Artists]

Justin Bieber 	 - 	 792 occurances
Juice WRLD 	 - 	 677 occurances
Billie Eilish 	 - 	 660 occurances
Ariana Grande 	 - 	 630 occurances
Pop Smoke 	 - 	 628 occurances
__________________________________________________________________________

[Top 5 Songs]

Falling 	 - 	 254 occurances
Levitating (feat. DaBaby) 	 - 	 144 occurances
Dynamite 	 - 	 144 occurances
Blinding Lights 	 - 	 144 occurances
DÁKITI 	 - 	 144 occurances
